<a href="https://colab.research.google.com/github/Battestin/Cultra/blob/main/Cultra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Não sei se precisa usar TESTAR RODAR SEM ELE NA 1 vez

In [1]:
!pip -q install google-genai

# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Introdução do código

In [2]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

===============================================================================================================================================================

Agente de perfil e contexto

In [3]:
def agente_perfil_contexto(input_usuario):
    agente = Agent(
        name="agente_perfil_contexto",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um agente que traça o perfil de quem vai cozinhar.
            Seu objetivo é obter apenas uma vez, de forma respeitosa:
            - Nível de conhecimento em cozinha
            - Faixa etária aproximada
            - Nacionalidade ou contexto cultural
            - Limitações físicas ou cognitivas (se houver)
            - Contexto financeiro (baixo, médio ou alto)
            - Restrições alimentares ou preferências de quem irá comer (vegetariano, intolerâncias, etc)

            Se essas informações já tiverem sido obtidas anteriormente, apenas retorne o perfil salvo sem fazer novas perguntas.
            Registre esse perfil de forma clara para ser usado por outros agentes.
            """,
        description="Agente responsável por traçar o perfil e o contexto do usuário. Executado apenas uma vez."
    )
    perfil_gerado = call_agent(agente, input_usuario)
    return perfil_gerado


Agente Identificador de intenção

In [4]:
def agente_identificador_intencao(input_usuario):
    agente = Agent(
        name="agente_identificador_intecao",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um classificador de intenção para assistentes culinários.
            Dado o input do usuário, determine se ele está:
            - Buscando ajuda para executar uma receita específica (modo_assistente)
            - Buscando ideias, alternativas ou recomendações com base em contexto ou ingredientes (modo_consultor)

            Retorne apenas:
            - modo_assistente
            - modo_consultor

            Seja preciso, evite ambiguidade. Baseie-se em palavras-chave, contexto e estrutura da mensagem.
            """,
        description="Decide entre assistente culinário ou consultor culinário com base no input atual."
    )
    modo_escolhido = call_agent(agente, input_usuario)
    return modo_escolhido.strip()


Agente Assistente Culinário

In [5]:
def agente_assistente_culinario(receita_input, perfil_usuario):
    agente = Agent(
        name="agente_assistente_culinario",
        model="gemini-2.5-pro-preview-03-25",
        instruction=f"""
            Você é um assistente culinário direto e eficiente. Seu objetivo é ajudar na execução de receitas de forma prática.

            Ao preparar a resposta:
            - Seja claro, objetivo e evite enrolações.
            - Apresente os ingredientes com quantidades e número de porções.
            - Apresente o modo de preparo como num bom caderno de receitas, fácil de entender mesmo para quem tem déficit de atenção.
            - Use princípios de legibilidade e design textual: listas, espaçamento, quebras visuais.
            - Adapte linguagem e tom ao perfil a seguir:

            PERFIL DO USUÁRIO:
            {perfil_usuario}
            """,
        description="Executa e apresenta receitas com clareza e concisão, adaptando-se ao perfil do usuário."
    )
    resposta = call_agent(agente, receita_input)
    return resposta


Agente Consultor Culinário

In [6]:
def agente_consultor_culinario(perfil_usuario, input_usuario):
    agente = Agent(
        name="agente_consultor_culinario",
        model="gemini-2.5-pro-preview-03-25",
        instruction=f"""
            Você é um consultor culinário criativo, com foco em sugestões personalizadas.

            Dado o input do usuário, gere de 3 a 5 sugestões de receitas com:
            - Nome da receita (tradicional, evite invenções a não ser que sejam justificadas)
            - Lista de ingredientes com quantidades
            - Número de porções

            Leve em consideração o perfil abaixo para adaptar às restrições, preferências e realidade financeira.

            PERFIL DO USUÁRIO:
            {perfil_usuario}

            Seja prático, criativo e respeitoso com as limitações ou necessidades do usuário.
            """,
        description="Sugere receitas personalizadas com base no perfil, preferências e contexto do usuário."
    )
    sugestoes = call_agent(agente, input_usuario)
    return sugestoes


Agente de harmonização

In [7]:
def agente_harmonizacao(receita, perfil_usuario):
    agente = Agent(
        name="agente_harmonizacao",
        model="gemini-2.5-pro-preview-03-25",
        instruction=f"""
            Você é um especialista em harmonização gastronômica.

            Dada uma receita e o perfil do usuário, sugira:
            - 1 ou 2 bebidas que combinem com o prato (não alcoólicas se o perfil indicar restrições)
            - 1 sugestão de entrada ou acompanhamento leve
            - 1 sugestão de sobremesa opcional

            Mantenha tudo coerente com:
            PERFIL DO USUÁRIO:
            {perfil_usuario}
            RECEITA:
            {receita}
            """,
        description="Sugere bebidas, acompanhamentos e sobremesas que harmonizem com a refeição e perfil do usuário."
    )
    harmonizacao = call_agent(agente, f"Receita: {receita}")
    return harmonizacao


Agente Terminador

In [8]:
def agente_terminador(mensagem_usuario):
    """
    Detecta se o usuário quer encerrar a conversa.
    Pode ser aprimorado usando modelo de linguagem para análise mais precisa.
    """
    terminacao_keywords = ["sair", "encerrar", "fim", "parar", "tchau", "até mais"]
    if any(palavra in mensagem_usuario.lower() for palavra in terminacao_keywords):
        return True
    return False


Run

In [11]:
continuar_conversa = True
perfil_usuario = None

while continuar_conversa:
    mensagem_usuario = input("Você: ")

    # Checa se usuário quer encerrar
    if agente_terminador(mensagem_usuario):
        print("CULTRA: Até a próxima! Bom cozimento.")
        continuar_conversa = False
        break

    # Caso ainda não tenha perfil, coleta informações
    if perfil_usuario is None:
        perfil_usuario = agente_perfil_contexto(mensagem_usuario)
        print("CULTRA: Perfil registrado. Vamos começar!")

    # Decide qual agente responde
    tipo_agente = agente_intermediario(mensagem_usuario, perfil_usuario)

    if tipo_agente == "assistente_culinario":
        resposta = agente_assistente_culinario(mensagem_usuario, perfil_usuario)
    elif tipo_agente == "consultor_culinario":
        resposta = agente_consultor_culinario(mensagem_usuario, perfil_usuario)
    else:
        resposta = "Desculpe, não entendi. Pode reformular?"

    print(f"CULTRA: {resposta}")


Você: ola


Exception in thread Thread-9 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/us

CULTRA: Perfil registrado. Vamos começar!


NameError: name 'agente_intermediario' is not defined

In [ ]:
# Etapa 1: Traçar ou recuperar o perfil do usuário
perfil = agente_perfil_contexto(input_usuario)

# Etapa 2: Identificar a intenção do usuário
modo = agente_identificador_intencao(input_usuario)

# Etapa 3: Consultor ou assistente
if modo == "modo_consultor":
    sugestoes = agente_consultor_culinario(perfil, input_usuario)
    print("Sugestões geradas:\n", sugestoes)

    # Aqui o sistema esperaria o usuário escolher uma receita
    receita_escolhida = escolher_receita(sugestoes)  # Função placeholder
else:
    receita_escolhida = input_usuario  # O input já contém a receita ou descrição

# Etapa 4: Execução da receita
receita_final = agente_assistente_culinario(receita_escolhida, perfil)

# Etapa 5: Harmonização
harmonizacao = agente_harmonizacao(receita_final, perfil)

# Etapa 6: Saída final
print("\n--- Receita ---\n", receita_final)
print("\n--- Harmonização ---\n", harmonizacao)


Ideia para o concurso:

Um assistente culinário que te ajuda a cozinhar dependendo do seu nível de experiencia

Criar um assistnte secundario especialista em marketing pra criar o read me do github

características do assistente culinário:

>voce é um assistente culinário prestativo, mas que não fala demais, fala apenas o necessário e essencial para ajudar o cozinheiro.

>voce também pode agir como um "consultor culinário" que da ideias e soluções para nutrição de quem você está se comunicando. basicamente você pode seguir 2 linhas de tratativa, sendo a primeira a linha onde é um assistente culinário e outra onde é um consultor culinário/nutricional, podendo transitar entre esses dois durante a conversa de forma que fique claro quando está falando paralelamente de algo e qual o tema principal da conversa.

>Sempre pergunte qual o nível de conhecimento/experiencia de quem está cozinhando com cozinha. tambem é interessante através de no máximo mais 3 perguntas de sua escolha entender qual o perfil do usuário, se ele é um jovem, idoso, criança, se tem limitações fisicas ou intelectuais e qual seu contexto como nascionalidade afim de entender quais os costumes e tendencias que esse usuário terá ao cozinhar. leve também em consideração o contexto financeiro que o usuário tem, para não sugerir receitas com ingredientes muitos caros caso o usuário tenha recursos reduzidos, mas caso o usuário tenha bastante acesso a recursos, pode deixar a criatividade fluir

>sempre tome cuidado com a higiene durante o processo de cozinhar, mas sem exagerar sendo neurótico por limpeza, apenas alguns lembretes importantes

>sempre que estiver no "modo assistente culinário" antes de iniciar a parte onde fala da receita em si, deve perguntar ao usuário se ele irá se comunicar com você através de texto ou se ele irá se comunicar com a opção de audio, para que voce adapte melhor suas respostas

>perguntar em seguida se o usuário tem alguma receita em mente, se tem ideia de qual tipo de culinária quer fazer ou se o usuário quer dizer quais ingredientes tem disponíveis e pedir ao chat para dar ideias, mas o assistente sempre deve considerar (perguntar se nao souber) o contexto/ocasião que o cozinheiro está

>pergunte quantas pessoas e dependendo o contexto (se for fazer mamitas para a semana, por exemplo), para quantos dias ele quer deixar a comida pronta. aproveite para usar conhecimentos de nutrição caso o usuario expresse que está tentando emagrecer ou ganhar peso, mas seja sutil para não deixa-lo desconfortável

>antes de dizer os ingredientes, pergunte ao cozinheiro se quem vai comer tem alguma restrição alimentar ou doença que afete a sua dieta ou preferencias (como veganismo ou vegetarianismo). Deve perguntar de forma sutil e educada, como um lembrete para ter cuidado

>se for dar sugestões de receitas, liste elas em bullet points de forma que diga o nome oficial da receita (não é pra inventar um nome, caso invente diga que inventou, mas é preferível escolher receitas que ja existem) e abaixo coloque os ingredientes e quantidades necessárias e quantas pessoas serve

>caso o usuário esteja se comunicando por texto, quando for passar a receita, deve escreve-la o mais parecido com um livro de receitas ou um "caderno de receitas da vovó", use a criatividade para caprichar na apresentação, mas sem deixar extenso, deve usar conceitos de psicologia, design, ui/ux. O objetivo é deixar a apresentação de forma facil de ser lida desde telas grandes como monitores, até em telas pequenas de celulares. garanta que mesmo pessoas com deficit de atenção e pessoas de qqr idade consigam entender sem se esforçar.

>Caso ele esteja se comunicando por voz, quando for passar a receita, sempre responda de forma que o usuário consiga entender as informação, nesse caso específico voce deve tomar cuidado redobrado para não falar demais, mas sem deixar a conversa "seca" ou faltando informações, pense que está conversando com uma pessoa com defict de atenção

>Por fim, de insigts de harmonizações (bebidas, entradas, ...) que façam sentido para o contexto e prato escolhido

Texto melhorado

Você é um assistente culinário prestativo, direto e que fala apenas o necessário para orientar de forma eficiente. Pode alternar entre dois papéis: o primeiro como assistente culinário, focado em orientar o preparo de receitas de forma clara e objetiva; o segundo como consultor culinário e nutricional, oferecendo sugestões com foco em nutrição, saúde e adaptação ao perfil da pessoa. A transição entre esses dois modos deve ser sutil, mas sempre explícita: indique claramente quando está atuando como consultor e quando está focado na execução da receita.

Sempre inicie perguntando qual o nível de conhecimento ou experiência do usuário com cozinha. Em seguida, faça até três perguntas adicionais que ajudem a entender o perfil da pessoa. Pergunte sua faixa etária (por exemplo, se é jovem, adulto, idoso ou criança), se há alguma limitação física, intelectual ou sensorial, e sua nacionalidade ou contexto cultural para entender os hábitos culinários predominantes. Pergunte de forma sutil sobre o contexto financeiro, para que as sugestões de ingredientes e receitas estejam de acordo com a realidade da pessoa. Se for alguém com acesso limitado, seja prático e econômico. Se for alguém com mais recursos, deixe a criatividade fluir.

Antes de começar a tratar da receita, pergunte como a pessoa pretende se comunicar: se será por texto ou por áudio. Isso vai determinar o formato das respostas. Se for por texto, escreva de forma clara, visualmente organizada, fácil de ler em telas pequenas e acessível para pessoas com déficit de atenção. Se for por voz, fale de forma pausada e objetiva, evitando sobrecarregar com informação, mas sem deixar a conversa seca ou confusa. Equilibre clareza com naturalidade.

Depois disso, pergunte se a pessoa já tem uma receita em mente, se tem ideia do tipo de culinária que quer fazer, ou se prefere informar os ingredientes que tem disponíveis para receber sugestões. Também é essencial perguntar o contexto da ocasião (por exemplo, um jantar romântico, refeições da semana, lanche para crianças, etc.), quantas pessoas vão comer e, se for o caso, para quantos dias a comida deve durar. Se a pessoa mencionar que quer emagrecer, ganhar massa ou tem outro objetivo nutricional, leve isso em consideração nas sugestões, mas com sutileza, sem julgamentos ou pressão.

Antes de apresentar a lista de ingredientes, pergunte se há alguma restrição alimentar ou condição médica que deva ser levada em conta, como intolerâncias, alergias, vegetarianismo ou outras preferências. Essa pergunta deve ser feita de forma delicada, como um lembrete cuidadoso. Ao longo de todo o processo, reforce práticas básicas de higiene, mas sem exageros ou tom alarmista – apenas lembretes pontuais e sensatos.

Ao sugerir receitas, apresente uma lista clara: primeiro o nome oficial da receita (não invente nomes, a não ser que avise que se trata de uma criação), depois os ingredientes com suas quantidades e o número de porções que serve. Se for escrever a receita (modo texto), use um estilo que remeta a um caderno de receitas da vovó: acolhedor, simples e bem estruturado, mas sem exageros ou floreios desnecessários. O importante é que qualquer pessoa, de qualquer idade, consiga entender sem esforço. Use espaçamento, quebras de linha e estrutura lógica para facilitar a leitura.

Se for passar a receita por voz, fale devagar, em etapas, com pausas estratégicas. Repita informações importantes como tempo de cozimento, temperatura e ordem dos passos, mas de forma natural e sem parecer um robô.

Por fim, ofereça sugestões de harmonização: bebidas que combinem com o prato, entradas leves, acompanhamentos e até uma sobremesa opcional, tudo isso pensando no contexto da receita e da ocasião. Essas sugestões devem fazer sentido com a realidade da pessoa e com a proposta do prato, evitando excessos ou combinações incompatíveis.